In [ ]:
df_problemes = pd.read_csv('./data/problems40000.csv')
df_problemes.drop('Unnamed: 0',axis=1,inplace=True)
df_problemes = df_problemes[df_problemes['recom_ratings'] != '[0, 0, 0]']
df_problemes.reset_index(drop=True, inplace=True)

In [ ]:
casos,_ = createDB(df_problemes)

In [ ]:
problems = [[int(df_problemes['user_id'][i])] + list(cas.descripcio) for i, cas in enumerate(casos)]

In [ ]:
# Definir las columnas del DataFrame
column_names = ['user_id','Fantasy', 'Fiction', 'Mystery', 'Poetry',
                'History', 'Romance', 'Non-fiction', 'Children', 'Young-adult',
                'Comics', 'mean_length', 'count_reads', 'avg_rating']

# Crear el DataFrame
df = pd.DataFrame(problems, columns=column_names)


In [1]:
from CBR import *
import numpy as np
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import ast

In [7]:
df

,user_id,Fantasy,Fiction,Mystery,Poetry,History,Romance,Non-fiction,Children,Young-adult,Comics,mean_length,count_reads,avg_rating
0,357865,0.8,0.6,0.7,0.833333,0.333333,1.0,0.4,0.5,0.1,0.2,0.102266,0.018421,0.327093
1,67954,0.8,0.7,0.6,1.000000,0.333333,0.9,0.4,0.5,0.2,0.1,0.093975,0.071382,0.773795
2,396177,0.7,0.1,0.8,1.000000,0.555556,0.9,0.3,0.2,0.6,0.4,0.104478,0.018421,0.157137
3,92823,0.8,0.6,0.7,1.000000,0.111111,0.9,0.5,0.3,0.2,0.4,0.103925,0.009868,0.202056
4,478254,0.2,0.5,0.7,1.000000,0.333333,0.8,0.4,0.9,0.0,0.6,0.127695,0.002632,0.713496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24294,722422,0.8,0.2,0.7,0.833333,0.333333,1.0,0.5,0.6,0.1,0.4,0.127142,0.007566,0.433734
24295,688967,0.8,0.5,0.6,0.833333,0.444444,1.0,0.1,0.7,0.3,0.2,0.131012,0.006908,0.106108
24296,400080,0.8,0.7,0.6,1.000000,0.333333,0.9,0.1,0.5,0.4,0.2,0.114981,0.093092,0.560940
24297,111592,0.8,0.6,0.7,1.000000,0.111111,0.9,0.2,0.5,0.4,0.3,0.118850,0.025658,0.787042


In [8]:
df_inter = pd.read_csv('D:\Escritorio\casebooks_SBC\interactions.csv') 
df_merged = df.merge(df_inter[['user_id', 'book_id', 'rating']], on='user_id', how='inner')

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\artur\AppData\Local\Temp\ipykernel_29204\2249414048.py:1: SyntaxWarning: invalid escape sequence '\E'
  df_inter = pd.read_csv('D:\Escritorio\casebooks_SBC\interactions.csv')
C:\Users\artur\AppData\Local\Temp\ipykernel_29204\2249414048.py:1: SyntaxWarning: invalid escape sequence '\E'
  df_inter = pd.read_csv('D:\Escritorio\casebooks_SBC\interactions.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Escritorio\\casebooks_SBC\\interactions.csv'

In [ ]:
bk_merge = pd.read_csv('data/genres_pages_x_books.csv')
bk_merge.drop('Unnamed: 0',axis=1,inplace=True)
final_dataset = df_merged.merge(bk_merge, on='book_id', how='inner')

In [ ]:
notna = final_dataset.dropna()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ajustar el tamaño de la figura a algo más grande
plt.figure(figsize=(20, 16))  # Tamaño más grande para mejor visualización

# Calcular la matriz de correlación
correlation_matrix = notna.corr()

# Crear el mapa de calor
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', annot_kws={"size":8})

# Ajustar el tamaño de la fuente de las anotaciones si es necesario
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
    
# Mostrar el gráfico
plt.show()


### Selecció de variables
Creem un random forest per veure la importància de les variables i treure les variables no importants

In [ ]:
X = notna.drop(['rating','user_id','book_id'], axis=1)
y = notna['rating']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Asumiendo que X e y ya están definidos y que y es una variable continua:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Random Forest para regresión
model = RandomForestRegressor(
    n_estimators=200,  # Número de árboles en el bosque.
    max_depth=20,  # Profundidad máxima del árbol.
    min_samples_split=2,  # Número mínimo de muestras requeridas para dividir un nodo.
    min_samples_leaf=1,  # Número mínimo de muestras requeridas para ser una hoja.
    max_features='sqrt',  # Número de características a considerar para cada división.
    max_leaf_nodes=None,  # Número máximo de nodos hoja.
    bootstrap=True,  # Utilizar muestras de bootstrap para construir árboles.
    random_state=42,
    n_jobs=-1,  # Número de trabajos en paralelo.
    oob_score=True  # Usar muestras fuera de la bolsa para estimar el error.
)

model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio del modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Error Cuadrático Medio (MSE) del modelo: {mse:.2f}')

# Calcular la importancia de las características
feature_importances = model.feature_importances_

In [ ]:
notna.drop(['rating','user_id','book_id'], axis=1).columns

In [ ]:
cols = list(notna.drop(['rating','user_id','book_id'], axis=1).columns)
print("Importancia de las características:")
non_important = []
for i, imp in enumerate(feature_importances):
    feat = cols[i]
    print(f'{feat}: {imp:.4f}')
    if imp < 0.01:
        X.drop(feat,axis=1,inplace=True)
        print('Variable eliminada')
        non_important.append(feat)

## Entrenem el model xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from xgboost.callback import TrainingCallback

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Clase para mostrar el progreso usando tqdm
class TQDMCallback(TrainingCallback):
    def __init__(self, total):
        self.pbar = tqdm(total=total)
    def after_iteration(self, model, epoch, evals_log):
        self.pbar.update(1)
        return False  # Return True para detener el entrenamiento
experiments = [(300,10),(500,6),(700,6),(300,20),(500,10)]

for params in experiments:
    # Instanciar el modelo XGBoost con early_stopping_rounds
    n_estimators, max_depth = params
    xgb_model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0,
        reg_alpha=0.1,
        reg_lambda=1,
        early_stopping_rounds=10
    )

    # Entrenar el modelo con barra de progreso
    xgb_model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)],
        callbacks=[TQDMCallback(n_estimators)],
        verbose=False   
    )

    # Predecir y evaluar
    predictions = xgb_model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    with open(f'xgb_{n_estimators}_{max_depth}.pkl', 'wb') as file:
        pickle.dump(xgb_model, file)
    print('Parametres:')
    print('n_estimators: ',n_estimators)
    print('max_depth: ',max_depth)
    print(rmse)

El paràmetre que millora la mse del model és el n_estimators. Per tant, provem augmentant més aquest paràmetre:

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from xgboost.callback import TrainingCallback

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Clase para mostrar el progreso usando tqdm
class TQDMCallback(TrainingCallback):
    def __init__(self, total):
        self.pbar = tqdm(total=total)
    def after_iteration(self, model, epoch, evals_log):
        self.pbar.update(1)
        return False  # Return True para detener el entrenamiento
experiments = [(1000,6),(1500,6)]

for params in experiments:
    # Instanciar el modelo XGBoost con early_stopping_rounds
    n_estimators, max_depth = params
    xgb_model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0,
        reg_alpha=0.1,
        reg_lambda=1,
        early_stopping_rounds=10
    )

    # Entrenar el modelo con barra de progreso
    xgb_model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)],
        callbacks=[TQDMCallback(n_estimators)],
        verbose=False   
    )

    # Predecir y evaluar
    predictions = xgb_model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    with open(f'xgb_{n_estimators}_{max_depth}.pkl', 'wb') as file:
        pickle.dump(xgb_model, file)
    print('Parametres:')
    print('n_estimators: ',n_estimators)
    print('max_depth: ',max_depth)
    print(rmse)

### Com aplicar a un nou cas

In [ ]:
non_important

In [ ]:
newp = Problem(1,0,0,0,0,0,0,0,0,0,0.5,0.5,0.5)
newcas = Cas(id=(2134214),descripcio=scale_features([newp])[0])
newcas.solucio = [89375,287142,1907184]

In [ ]:
def convert_case_to_data(newcas):
    non_important = ['Fantasy_x', 'Poetry_x']
    newcas = [newcas]
    problems = [list(cas.descripcio) + [str(cas.solucio)] for i, cas in enumerate(newcas)]
    # Definir las columnas del DataFrame
    column_names = ['Fantasy', 'Fiction', 'Mystery', 'Poetry',
                    'History', 'Romance', 'Non-fiction', 'Children', 'Young-adult',
                    'Comics', 'mean_length', 'count_reads', 'avg_rating','books_ids']

    # Crear el DataFrame
    df = pd.DataFrame(problems, columns=column_names)
    df['books_ids'] = df['books_ids'].apply(eval)
    # Utilizar explode para expandir los elementos de la lista en filas separadas
    df_exploded = df.explode('books_ids')
    # Cambiar el nombre de la columna 'lista' a 'nuevo_nombre'
    df_exploded = df_exploded.rename(columns={'books_ids': 'book_id'})
    
    bk_merge = pd.read_csv('data/genres_pages_x_books.csv')
    bk_merge.drop('Unnamed: 0',axis=1,inplace=True)
    final_dataset = df_exploded.merge(bk_merge, on='book_id', how='inner')
    final_dataset.drop(non_important,axis=1,inplace=True)
    final_dataset.drop(['book_id'],axis=1,inplace=True)
    return final_dataset

In [ ]:
dataset = convert_case_to_data(newcas)

In [ ]:
dataset.columns

In [ ]:
def estimacio_avaluacio(newcas,model_path): # Se pot canviar el paràmetre per un path fixe si decidim quedarnos amb un model
    with open(model_path, 'rb') as file:
        loaded_model = pickle.load(file)
    dataset = convert_case_to_data(newcas)
    estimacio = np.clip(np.round(np.array(loaded_model.predict(dataset))), 0, 5)
    return estimacio.tolist()

In [5]:
estimacio_avaluacio(newcas,'xgb_1500_6.pkl')

/home/oscarmolina/.venv/sbc/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [00:34:41] WARNING: /workspace/src/common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


NameError: name 'df_problemes' is not defined